# AnimeLSTM
Basically what I'm trying to do is train a NN to learn to play anime piano music. All training data is in ./data and bellow I encode each note in every song into an array (if its a chord I encode the chord into a string speparated by "."). Then I'll use an LSTM to guess what chord comes next after letting it predict it based on ~100 past notes.

In [111]:
# imports
from tqdm.notebook import tqdm
import numpy as np
import glob
from time import time
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization as BatchNorm
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from music21 import converter, instrument, note, chord, stream

In [123]:
notes = []
allFiles = glob.glob("data/*.mid")
for file in tqdm(allFiles):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for sound in notes_to_parse:
        if isinstance(sound, note.Note):
            notes.append(str(sound.pitch))
        elif isinstance(sound, chord.Chord):
            notes.append('.'.join(str(n) for n in sound.normalOrder))

Save the notes in notes.npy and turn into numpy array for easy use later on.

In [127]:
notes = np.array(notes)
np.save("notes.npy",notes)

Parse the data into LSTM compatible data (uses past 100 notes to predict next 1)

In [126]:
sequence_length = 300

# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
non_normalized_input = network_input
network_input = network_input / float(len(pitchnames))
network_output = to_categorical(network_output)

In [129]:
np.save("x.npy", network_input)
np.save("y.npy", network_output)

In [130]:
model = Sequential()
model.add(LSTM(
    400,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(800, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(400))
model.add(Dense(400))
model.add(Dropout(0.3))
model.add(Dense(len(pitchnames)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights\weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
) 
tensorboard = TensorBoard(log_dir="logs\{}".format(time()), profile_batch=0, update_freq='epoch')
callbacks_list = [checkpoint, tensorboard]     
model.fit(network_input, network_output, epochs=300, batch_size=64, callbacks=callbacks_list)

Train on 77735 samples
Epoch 1/300


19968/77735 [======>.......................] - ETA: 34:16 - loss: 5.65 - ETA: 34:10 - loss: 5.61 - ETA: 34:24 - loss: 5.77 - ETA: 34:24 - loss: 5.66 - ETA: 34:19 - loss: 5.54 - ETA: 34:14 - loss: 5.45 - ETA: 34:11 - loss: 5.37 - ETA: 34:08 - loss: 5.33 - ETA: 34:05 - loss: 5.30 - ETA: 34:02 - loss: 5.27 - ETA: 34:00 - loss: 5.24 - ETA: 33:58 - loss: 5.22 - ETA: 33:56 - loss: 5.21 - ETA: 33:54 - loss: 5.19 - ETA: 33:52 - loss: 5.17 - ETA: 33:51 - loss: 5.17 - ETA: 33:49 - loss: 5.16 - ETA: 33:46 - loss: 5.15 - ETA: 33:45 - loss: 5.13 - ETA: 33:43 - loss: 5.11 - ETA: 33:41 - loss: 5.10 - ETA: 33:39 - loss: 5.09 - ETA: 33:38 - loss: 5.09 - ETA: 33:36 - loss: 5.07 - ETA: 33:34 - loss: 5.07 - ETA: 33:32 - loss: 5.06 - ETA: 33:31 - loss: 5.05 - ETA: 33:30 - loss: 5.05 - ETA: 33:28 - loss: 5.04 - ETA: 33:27 - loss: 5.03 - ETA: 33:27 - loss: 5.03 - ETA: 33:26 - loss: 5.02 - ETA: 33:25 - loss: 5.02 - ETA: 33:23 - loss: 5.01 - ETA: 33:21 - loss: 5.01 - ETA: 33:20 - loss: 5.01 - ETA: 33:21 - loss

40064/77735 [==============>...............] - ETA: 25:59 - loss: 4.86 - ETA: 25:57 - loss: 4.87 - ETA: 25:56 - loss: 4.86 - ETA: 25:54 - loss: 4.86 - ETA: 25:52 - loss: 4.86 - ETA: 25:51 - loss: 4.86 - ETA: 25:49 - loss: 4.86 - ETA: 25:47 - loss: 4.86 - ETA: 25:46 - loss: 4.86 - ETA: 25:44 - loss: 4.86 - ETA: 25:42 - loss: 4.86 - ETA: 25:40 - loss: 4.86 - ETA: 25:39 - loss: 4.86 - ETA: 25:37 - loss: 4.86 - ETA: 25:35 - loss: 4.86 - ETA: 25:33 - loss: 4.86 - ETA: 25:32 - loss: 4.86 - ETA: 25:30 - loss: 4.86 - ETA: 25:28 - loss: 4.86 - ETA: 25:26 - loss: 4.86 - ETA: 25:25 - loss: 4.86 - ETA: 25:23 - loss: 4.86 - ETA: 25:21 - loss: 4.86 - ETA: 25:20 - loss: 4.86 - ETA: 25:18 - loss: 4.86 - ETA: 25:16 - loss: 4.86 - ETA: 25:14 - loss: 4.86 - ETA: 25:13 - loss: 4.86 - ETA: 25:11 - loss: 4.86 - ETA: 25:09 - loss: 4.86 - ETA: 25:07 - loss: 4.86 - ETA: 25:06 - loss: 4.86 - ETA: 25:04 - loss: 4.86 - ETA: 25:02 - loss: 4.86 - ETA: 25:00 - loss: 4.86 - ETA: 24:59 - loss: 4.86 - ETA: 24:57 - loss

60160/77735 [======================>.......] - ETA: 17:01 - loss: 4.82 - ETA: 16:59 - loss: 4.82 - ETA: 16:58 - loss: 4.82 - ETA: 16:56 - loss: 4.82 - ETA: 16:54 - loss: 4.82 - ETA: 16:53 - loss: 4.82 - ETA: 16:51 - loss: 4.82 - ETA: 16:49 - loss: 4.82 - ETA: 16:48 - loss: 4.82 - ETA: 16:46 - loss: 4.82 - ETA: 16:44 - loss: 4.82 - ETA: 16:42 - loss: 4.82 - ETA: 16:41 - loss: 4.82 - ETA: 16:39 - loss: 4.82 - ETA: 16:37 - loss: 4.82 - ETA: 16:35 - loss: 4.82 - ETA: 16:34 - loss: 4.82 - ETA: 16:32 - loss: 4.82 - ETA: 16:30 - loss: 4.82 - ETA: 16:28 - loss: 4.82 - ETA: 16:27 - loss: 4.82 - ETA: 16:25 - loss: 4.82 - ETA: 16:23 - loss: 4.82 - ETA: 16:21 - loss: 4.82 - ETA: 16:20 - loss: 4.82 - ETA: 16:18 - loss: 4.82 - ETA: 16:16 - loss: 4.82 - ETA: 16:14 - loss: 4.82 - ETA: 16:13 - loss: 4.82 - ETA: 16:11 - loss: 4.82 - ETA: 16:09 - loss: 4.82 - ETA: 16:08 - loss: 4.82 - ETA: 16:06 - loss: 4.82 - ETA: 16:04 - loss: 4.82 - ETA: 16:02 - loss: 4.82 - ETA: 16:01 - loss: 4.82 - ETA: 15:59 - loss

77735/77735 [==============================] - ETA: 7:56 - loss: 4.796 - ETA: 7:54 - loss: 4.796 - ETA: 7:52 - loss: 4.796 - ETA: 7:51 - loss: 4.796 - ETA: 7:49 - loss: 4.796 - ETA: 7:47 - loss: 4.796 - ETA: 7:45 - loss: 4.796 - ETA: 7:44 - loss: 4.796 - ETA: 7:42 - loss: 4.796 - ETA: 7:40 - loss: 4.796 - ETA: 7:38 - loss: 4.796 - ETA: 7:37 - loss: 4.796 - ETA: 7:35 - loss: 4.796 - ETA: 7:33 - loss: 4.796 - ETA: 7:32 - loss: 4.796 - ETA: 7:30 - loss: 4.796 - ETA: 7:28 - loss: 4.796 - ETA: 7:26 - loss: 4.796 - ETA: 7:25 - loss: 4.796 - ETA: 7:23 - loss: 4.795 - ETA: 7:21 - loss: 4.795 - ETA: 7:19 - loss: 4.795 - ETA: 7:18 - loss: 4.795 - ETA: 7:16 - loss: 4.795 - ETA: 7:14 - loss: 4.795 - ETA: 7:12 - loss: 4.795 - ETA: 7:11 - loss: 4.795 - ETA: 7:09 - loss: 4.795 - ETA: 7:07 - loss: 4.795 - ETA: 7:05 - loss: 4.795 - ETA: 7:04 - loss: 4.795 - ETA: 7:02 - loss: 4.795 - ETA: 7:00 - loss: 4.795 - ETA: 6:58 - loss: 4.795 - ETA: 6:57 - loss: 4.795 - ETA: 6:55 - loss: 4.795 - ETA: 6:53 - loss:

## Generation

In [119]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = non_normalized_input[start]
prediction_output = []
# generate 500 notes
for note_index in tqdm(range(1000)):
    prediction_input = np.reshape(pattern,(1,pattern.shape[0],1))
    prediction_input = prediction_input / float(len(pitchnames))
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = np.append(pattern,[index])
    pattern = pattern[1:len(pattern)]

In [120]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [121]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp=f'anime_music_{time()}.mid')

'anime_music_1589394248.6651573.mid'